In [111]:
import numpy as np
import networkx as nx
import karateclub as kc
from sklearn.metrics import roc_auc_score
import math

In [122]:
from itertools import combinations
from typing import Any, Callable, Dict, Iterator, Tuple

ScoreFunc = Callable[[int, int, nx.Graph], float]


def preferential_attachment(n: int, m: int, graph: nx.Graph) -> float:
    return graph.degree(n) * graph.degree(m)


def common_neighbors(n: int, m: int, graph: nx.Graph) -> float:
    return len(set(graph.neighbors(n)) & set(graph.neighbors(m)))


def jacquard(n: int, m: int, graph: nx.Graph) -> float:
    n_n, n_m = set(graph.neighbors(n)), set(graph.neighbors(m))
    return len(n_n & n_m) / len(n_n | n_m)


def adamic_adar(n: int, m: int, graph: nx.Graph) -> float:
    neighbors = set(graph.neighbors(n)) & set(graph.neighbors(m))
    return sum(
        1 / math.log(graph.degree(k)) if graph.degree(k) > 1 else 0 for k in neighbors
    )


def random(n: int, m: int, graph: nx.Graph) -> float:
    return np.random.uniform(0, 1)


def missing_links(graph: nx.Graph) -> Iterator[Tuple[int, int]]:
    return filter(lambda ns: not graph.has_edge(*ns), combinations(graph.nodes, 2))


def evaluate(
    graph: nx.Graph,
    score: ScoreFunc,
    true_links: np.ndarray,
    k: int = 8,
    T: int = 1,
    print_links: bool = False,
    kwargs_func: Callable[[nx.Graph], Dict[str, Any]] = lambda g: {},
):
    top_score = acc = auroc = 0
    for _ in range(T):
        missing = np.array(list(missing_links(graph)))

        kwargs = kwargs_func(graph)
        scores = np.array([(n, m, score(n, m, graph, **kwargs)) for n, m in missing])

        # top_idxs = np.argpartition(scores[:, 2], -k)[-k:]
        top_idxs = np.argsort(scores[:, 2])[::-1][:k]

        if print_links:
            print("Links:")
            print(missing[top_idxs] + 1)

        top_score += scores[top_idxs].sum()
        acc += true_links[top_idxs].sum()
        auroc += roc_auc_score(true_links, scores[:, 2])

    name = score.__name__
    if name.endswith("_score"):
        name = name[: -len("_score")]
    print(name)
    if T > 1:
        print("Averaged over", T, "trials")
    print("Top", k, "Score:", top_score / T)
    print("Accuracy:", acc / T, "/", k)
    print("AUROC:", auroc / T)
    print()

In [113]:
# Zero index karate club graph
graph = nx.karate_club_graph()
rm_links = [
    (i - 1, j - 1)
    for i, j in [
        (1, 5),
        (2, 4),
        (3, 29),
        (6, 17),
        (9, 34),
        (16, 33),
        (24, 26),
        (25, 32),
    ]
]
graph.remove_edges_from(rm_links)
true_links = np.array([(n, m) in rm_links for n, m in missing_links(graph)])

In [ ]:
score_funcs = [preferential_attachment, common_neighbors, jacquard, adamic_adar]

evaluate(graph, random, true_links, T=100)
for score in score_funcs:
    evaluate(graph, score, true_links)

In [134]:
import dcsbm
import latent_space_model as lsm


# Degree-Corrected Stochastic Block Model
def dcsbm_params(graph: nx.Graph) -> Dict[str, Any]:
    A = nx.to_numpy_array(graph, weight=None) - np.identity(graph.number_of_nodes())
    g = dcsbm.regularized_spectral_clustering(A, 2)
    _, P = dcsbm.parameter_estimation(A, g)
    return {"g": g, "P": P}


def dcsbm_score(
    n: int, m: int, graph: nx.Graph, g: np.ndarray = None, P: np.ndarray = None
) -> float:
    return P[g[n], g[m]]


# Latent Space Model
def lsm_params(graph: nx.Graph) -> Dict[str, Any]:
    A = nx.to_numpy_array(graph, weight=None) - np.identity(graph.number_of_nodes())
    X, a, L, res = lsm.estimateParams(A, dim=2)
    return {"X": X, "a": a}


def lsm_score(
    n: int, m: int, graph: nx.Graph, X: np.ndarray = None, a: float = 0
) -> float:
    return a - np.linalg.norm(X[n] - X[m])


# Deep Walk
def deep_walk_embeddings(graph: nx.Graph) -> Dict[str, Any]:
    deep_walk = kc.DeepWalk(dimensions=2)
    deep_walk.fit(graph)
    return {"x": deep_walk.get_embedding()}


def deep_walk_score(n: int, m: int, graph: nx.Graph, x: np.ndarray = None) -> float:
    return x[n] @ x[m]

In [135]:
score_funcs = [(dcsbm_params, dcsbm_score), (lsm_params, lsm_score), (deep_walk_embeddings, deep_walk_score)]

evaluate(graph, random, true_links, T=100)
for params, score in score_funcs:
    evaluate(graph, score, true_links, k=10, T=5, kwargs_func=params)

random
Averaged over 100 trials
Top 8 Score: 273.11292757286634
Accuracy: 0.18 / 8
AUROC: 0.4980615942028985

dcsbm
Averaged over 5 trials
Top 10 Score: 462.0886611717532
Accuracy: 0.0 / 10
AUROC: 0.661620082815735

Initializing latent node positions by multidimensional scaling


C:\Users\Aaron\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
C:\Users\Aaron\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
C:\Users\Aaron\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set

Iteratively maximizing likelihood using quasi-Newton approach
Initializing latent node positions by multidimensional scaling
Iteratively maximizing likelihood using quasi-Newton approach


C:\Users\Aaron\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\manifold\_mds.py:298: FutureWarning: The default value of `normalized_stress` will change to `'auto'` in version 1.4. To suppress this warning, manually set the value of `normalized_stress`.
  warnings.warn(


Initializing latent node positions by multidimensional scaling
Iteratively maximizing likelihood using quasi-Newton approach


...

Initializing latent node positions by multidimensional scaling
Iteratively maximizing likelihood using quasi-Newton approach


C:\Users\Aaron\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\manifold\_mds.py:298: FutureWarning: The default value of `normalized_stress` will change to `'auto'` in version 1.4. To suppress this warning, manually set the value of `normalized_stress`.
  warnings.warn(


Initializing latent node positions by multidimensional scaling
Iteratively maximizing likelihood using quasi-Newton approach


C:\Users\Aaron\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\manifold\_mds.py:298: FutureWarning: The default value of `normalized_stress` will change to `'auto'` in version 1.4. To suppress this warning, manually set the value of `normalized_stress`.
  warnings.warn(


lsm
Averaged over 5 trials
Top 10 Score: 360.6189439499583
Accuracy: 0.6 / 10
AUROC: 0.762888198757764

deep_walk
Averaged over 5 trials
Top 10 Score: 547.2291046142578
Accuracy: 2.0 / 10
AUROC: 0.6880952380952381

